In [76]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import load_boston, load_iris, load_wine, load_digits, \
    load_breast_cancer, load_diabetes, fetch_mldata
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import itertools

In [2]:
df = pd.read_csv("rent10k.csv")
df.head(2)

,bedrooms,bathrooms,latitude,longitude,interest_level,hells,astoria,Evillage,Wvillage,LowerEast,...,ParkSlope,Prospect Park,Crown Heights,financial,brooklynheights,gowanus,num_photos,num_desc_words,num_features,price
0,4,2.0,40.7360,-74.0009,1,34.7,122.9796,28.906832,2.89,37.5946,...,87.433,374.45,138.906392,36.936126,47.461758,66.9,6,68,0,8995
1,0,1.0,40.7064,-74.0100,1,73.4,161.6796,41.934380,35.81,34.3606,...,66.933,394.95,118.406392,6.902838,26.961758,46.4,15,0,1,2880


In [67]:
def fit(X,y,order=2):
    "Pass X=[x1,x2] to get nth order polynomial with interaction terms"
    poly = PolynomialFeatures(order, interaction_only=False)
    model = make_pipeline(poly, Ridge())
    model.fit(X, y)
    ridge = model.named_steps['ridge']
    # y_pred = model.predict(x)
    # ax.plot(x, y_pred, ':', c='k', lw=.7)
    terms = poly.get_feature_names()
    terms[0] = 'c'
#     terms = reversed(terms)
    terms = [f'{c:.4f}{t}' for c,t in zip(ridge.coef_, terms)]
#     print(list(zip(ridge.coef_, terms)))
    eqn = ' + '.join( terms )
    return model, eqn

In [68]:
X = df.drop('price',axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X = X_train[['bedrooms','bathrooms']]
model, eqn = fit(X_train[['bedrooms','bathrooms']], y_train)

In [69]:
y_pred = model.predict(X_train[['bedrooms','bathrooms']])
mean_absolute_error(y_train, y_pred), mean_squared_error(y_train, y_pred), r2_score(y_train, y_pred)

(745.4976537788123, 1060592.1569188016, 0.4809857569555598)

In [70]:
y_pred = model.predict(X_test[['bedrooms','bathrooms']])
mean_absolute_error(y_test, y_pred), mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred)

(712.6256751434294, 955527.0686508613, 0.4686303028598061)

In [126]:
allpairs = list(itertools.combinations(range(X_train.shape[1]), 2))

features = X_train.columns.values

pairs = []
r2_trains, mae_trains = [], []
r2_tests, mae_tests = [], []
for pair in allpairs:
    pairs.append(features[[pair[0],pair[1]]])
#     print(pair)
    model, eqn = fit(X_train[features[[pair[0],pair[1]]]], y_train)
    y_pred = model.predict(X_train[['bedrooms','bathrooms']])
    r2_train, mae_train = r2_score(y_train, y_pred), mean_absolute_error(y_train, y_pred)
    r2_trains.append(r2_train)
    mae_trains.append(mae_train)
    y_pred = model.predict(X_test[['bedrooms','bathrooms']])
    r2_test, mae_test = r2_score(y_test, y_pred), mean_absolute_error(y_test, y_pred)
    r2_tests.append(r2_test)
    mae_tests.append(mae_test)

pairs = np.array(pairs)
r2_tests = np.array(r2_tests)
mae_tests = np.array(mae_tests)
r2_trains = np.array(r2_trains)
mae_trains = np.array(mae_trains)

# pick best k per generation
k = 5
best_idx = np.argsort(mae_tests)
best_idx = best_idx[:k]
info = list(zip(pairs[best_idx], r2_tests[best_idx], mae_tests[best_idx]))

for pair, r2, mae in info:
    print(pair, r2, mae)

['bedrooms' 'bathrooms'] 0.4686303028598061 712.6256751434294
['bedrooms' 'num_photos'] 0.3548850337952293 780.8201166913007
['bedrooms' 'num_desc_words'] 0.3414093235453868 782.1963908141901
['bedrooms' 'ParkSlope'] 0.34049620126320324 791.7970331711559
['bedrooms' 'num_features'] 0.26895400287837956 817.2126739767635
